# 历史消息记录的持久化存储

在许多情况下，最好保留对话历史记录。 `RunnableWithMessageHistory` 不知道 `get_session_history` 回调函数如何检索其聊天消息历史记录。以下是两个例子：
- [本地文件系统](https://github.com/langchain-ai/langserve/blob/main/examples/chat_with_persistence_and_user/server.py)
- [Redis](https://python.langchain.com/docs/expression_language/how_to/message_history/)

In [1]:
REDIS_URL = "redis://192.168.56.101:6379/0"

In [4]:
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI

# 定义对话链
model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | model

def get_message_history(session_id: str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(session_id, url=REDIS_URL)


with_message_history = RunnableWithMessageHistory(
    runnable,
    get_message_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [5]:
with_message_history.invoke(
    {"ability": "math", "input": "What does cosine mean?"},
    config={"configurable": {"session_id": "foobar"}},
)

AIMessage(content='Cosine is a trigonometric function that represents the ratio of the adjacent side to the hypotenuse in a right triangle.', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 33, 'total_tokens': 59}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-53c89928-60d0-45f5-bca8-dafdea8ed1ac-0')

In [6]:
with_message_history.invoke(
    {"ability": "math", "input": "What's its inverse"},
    config={"configurable": {"session_id": "foobar"}},
)

AIMessage(content='The inverse of cosine is called arccosine or inverse cosine. It gives the angle whose cosine is a given value.', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 71, 'total_tokens': 96}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c9da1292-a039-4bcb-854e-539734284efc-0')